In [1]:
import pandas as pd
import numpy as np

In [2]:
# file = open('data/train.bin', 'rb')
# byte = file.read(1)
# while byte: 
#     print(byte)
#     byte = file.read(1)
# file.close()

# with open('data/test.bin', mode='rb') as file: # b is important -> binary
#     fileContent = file.read()

In [3]:
# dt = np.dtype([('article', np.unicode_), ('highlights', np.unicode_)])
# data = np.fromfile('data/test.bin', dtype=dt)
# df = pd.DataFrame(data)

### Reading the debateall dataset

In [4]:
data = pd.read_csv('data/debateall.csv')
data.head()

,Full-Document,Citation,Extract,Abstract,#CharsDocument,#CharsAbstract,#CharsExtract,#WordsDocument,#WordsAbstract,#WordsExtract,AbsCompressionRatio,ExtCompressionRatio,OriginalDebateFileName
0,German Chancellor Angela Merkel demanded on Th...,O’Donnell and Baker ’13 [JOHN O'DONNELL AND LU...,"Merkel demanded the U S strike a ""no-spying"" a...",Text: The United States should offer to accede...,3155,132,1416,506,22,231,0.043478,0.456522,Bulk Data Collection Negative - JDI 2015.html5
1,Judge Leon last week questioned the effectiven...,"Cohen ’13 [ANDREW COHEN, DEC 27, 2013, The Atl...",Pauley asserted The effectiveness of bulk tele...,Unique link- judicial deference specifically o...,367,132,100,52,22,13,0.423077,0.250000,Bulk Data Collection Negative - JDI 2015.html5
2,New York Times July 25th 2013\n(POLITICS Rober...,Rosenthal 6/12/15\n(“Government's Secret Surve...,The recent leaks about government spying progr...,Issues of national surveillance are a key issu...,10386,61,2223,1639,10,355,0.006101,0.216595,Bulk Data Collection Negative - JDI 2015.html5
3,Posner 2007\n(“Terror in the Balance: Security...,New York Times July 25th 2013\n(POLITICS Rober...,"When national emergencies strike, the executiv...",Chief Justice Roberts ensures issues of nation...,1598,99,669,226,14,98,0.061947,0.433628,Bulk Data Collection Negative - JDI 2015.html5
4,Goitien April 17th 2015\n(Appointing Democrati...,Posner 2007\n(“Terror in the Balance: Security...,Chief Justice Roberts recently named two new j...,Courts empirically defer instances of national...,7696,56,1482,1201,7,238,0.005828,0.198168,Bulk Data Collection Negative - JDI 2015.html5


In [5]:
data.loc[0, 'Full-Document']

'German Chancellor Angela Merkel demanded on Thursday that the United States strike a "no-spying" agreement with Berlin and Paris by the end of the year, saying alleged espionage against two of Washington\'s closest EU allies had to be stopped.¶ Speaking after talks with EU leaders that were dominated by allegations that the U.S. National Security Agency had accessed tens of thousands of French phone records and monitored Merkel\'s private mobile phone, the chancellor said she wanted action from President Barack Obama, not just apologetic words.¶ Germany and France would seek a "mutual understanding" with the United States on cooperation between their intelligence agencies, and other EU member states could eventually take part.¶ "That means a framework for cooperation between the relevant (intelligence) services.Germany and France have taken the initiative and other member states will join," she said.¶ In a statement issued after the first day of the summit, the EU\'s 28 leaders said t

In [6]:
data.loc[0, 'Extract']

'Merkel demanded the U S strike a "no-spying" agreement with Berlin and Paris saying alleged espionage against EU allies had to be stopped the chancellor said she wanted action from Obama Germany and France would seek a "mutual understanding" with the United States on cooperation between their intelligence agencies, and other EU member states could eventually take part the EU\'s 28 leaders said they supported the Franco-German plan Merkel raised the with Obama but nothing came of it The United States has a "no-spying" deal with Britain, Australia, New Zealand and Canada, an alliance known as "Five Eyes" there has traditionally been a reluctance to make similar arrangements with other allies Merkel said an accord with Washington was long overdue The friendship and partnership between the European member states, includingGermany, and the United States is not a one-way street. We depend on it the United States also needs friends in the w As EU leaders arrived for the two-day summit there 

### Learning how to generate embeddings using BERT

In [7]:
# !pip install simpletransformers

In [8]:
from simpletransformers.language_representation import RepresentationModel

In [18]:
sentences = data['Full-Document'][: 1000]
sentences.head()

0    German Chancellor Angela Merkel demanded on Th...
1    Judge Leon last week questioned the effectiven...
2    New York Times July 25th 2013\n(POLITICS Rober...
3    Posner 2007\n(“Terror in the Balance: Security...
4    Goitien April 17th 2015\n(Appointing Democrati...
Name: Full-Document, dtype: object

In [72]:
summaries = data['Extract'][: 1000]
summaries.head()

0    Merkel demanded the U S strike a "no-spying" a...
1    Pauley asserted The effectiveness of bulk tele...
2    The recent leaks about government spying progr...
3    When national emergencies strike, the executiv...
4    Chief Justice Roberts recently named two new j...
Name: Extract, dtype: object

### Defining a model to use the bert embeddings and set use_cuda = True to use GPU

In [19]:
# sentences = ['Machine Learning and Deep Learning are part of AI', 'Data Science will excel in future']
model = RepresentationModel(
        model_type='bert',
        model_name='bert-base-uncased',
        use_cuda=True
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTextRepresentation: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Creating BERT embeddings for our dataset

In [26]:
word_vectors = model.encode_sentences(sentences, combine_strategy='mean')

In [33]:
word_vectors

array([[-0.32104653,  0.09997991,  0.04376634, ..., -0.10236427,
         0.03420955, -0.06911948],
       [-0.02158269, -0.10440352,  0.02366636, ..., -0.01730064,
         0.02124303, -0.02405714],
       [-0.00200591,  0.07882713,  0.11912461, ..., -0.13744695,
         0.04842247, -0.02859216],
       ...,
       [-0.26859814,  0.28943413, -0.09297414, ..., -0.19208944,
         0.01625864, -0.0901525 ],
       [-0.04423646,  0.07509145,  0.01463317, ..., -0.32493857,
        -0.06378958,  0.1337055 ],
       [ 0.05446831,  0.27189332,  0.1912251 , ..., -0.21712351,
         0.11477803,  0.0298214 ]], dtype=float32)

### Running K-Means clustering on the embeddings and finding the optimal K

In [38]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

distortions = []
inertias = []
mapping1 = {}
mapping2 = {}
for i in range(1, 101):
    print(f'\rRunning for k = {i} cluster', sep='')
    kMeans = KMeans(n_clusters=i, random_state=42).fit(word_vectors)
    distortions.append(sum(np.min(cdist(word_vectors, kMeans.cluster_centers_,
                                        'euclidean'), axis=1)) / word_vectors.shape[0])
    inertias.append(kMeans.inertia_)
 
    mapping1[i] = sum(np.min(cdist(word_vectors, kMeans.cluster_centers_,
                                   'euclidean'), axis=1)) / word_vectors.shape[0]
    mapping2[i] = kMeans.inertia_

Running for k = 1 cluster


C:\Users\ashut\anaconda3\envs\py\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  f"KMeans is known to have a memory leak on Windows "


Running for k = 2 cluster
Running for k = 3 cluster
Running for k = 4 cluster
Running for k = 5 cluster
Running for k = 6 cluster
Running for k = 7 cluster
Running for k = 8 cluster
Running for k = 9 cluster
Running for k = 10 cluster
Running for k = 11 cluster
Running for k = 12 cluster
Running for k = 13 cluster
Running for k = 14 cluster
Running for k = 15 cluster
Running for k = 16 cluster
Running for k = 17 cluster
Running for k = 18 cluster
Running for k = 19 cluster
Running for k = 20 cluster
Running for k = 21 cluster
Running for k = 22 cluster
Running for k = 23 cluster
Running for k = 24 cluster
Running for k = 25 cluster
Running for k = 26 cluster
Running for k = 27 cluster
Running for k = 28 cluster
Running for k = 29 cluster
Running for k = 30 cluster
Running for k = 31 cluster
Running for k = 32 cluster
Running for k = 33 cluster
Running for k = 34 cluster
Running for k = 35 cluster
Running for k = 36 cluster
Running for k = 37 cluster
Running for k = 38 cluster
Running f

In [39]:
for key, val in mapping1.items():
    print(f'{key} : {val}')

1 : 3.6570092134246925
2 : 3.459906687787475
3 : 3.4102918187867197
4 : 3.332789104884604
5 : 3.286157217508886
6 : 3.247205859804996
7 : 3.2214122136523016
8 : 3.196712911866378
9 : 3.1742071010481467
10 : 3.1528242857508584
11 : 3.13409939097311
12 : 3.1133796535911173
13 : 3.096785313801534
14 : 3.0802580628411054
15 : 3.0604947274833463
16 : 3.0553298238488606
17 : 3.0428161465673513
18 : 3.030522122099931
19 : 3.0217885733608214
20 : 3.016082845934434
21 : 3.0066811024839506
22 : 3.0012597593741246
23 : 2.9862032247411308
24 : 2.981111650588591
25 : 2.9806670674727753
26 : 2.971396830889518
27 : 2.9646291326463854
28 : 2.952725127970656
29 : 2.9470008019970426
30 : 2.942999875705094
31 : 2.9359836072329335
32 : 2.9313626411117215
33 : 2.9256194393101977
34 : 2.9186749028936183
35 : 2.9121242026120404
36 : 2.903320775221513
37 : 2.900125380466896
38 : 2.888238750373877
39 : 2.886579524264304
40 : 2.8827074007260722
41 : 2.8807329318007375
42 : 2.871822238572835
43 : 2.8717494090362

### Getting the cluster centers and the predicted labels

In [44]:
preds = kMeans.labels_
centers = kMeans.cluster_centers_

### Creating a hashMap for the points in each of the cluster to find the closest point

In [60]:
from collections import defaultdict

clusters = defaultdict(list)
for i in range(1000):
    if preds[i] <= 99:
        clusters[preds[i]].append([centers[preds[i]], i, word_vectors[i]])

### Finding the closest point in each of selected cluster

In [68]:
cluster_repr = {}
for center, points in clusters.items():
    dists = []
    for i, point in enumerate(points):
        dists.append([np.linalg.norm(center - point[2]), i, point[2]])
    dists.sort(key=lambda x: x[0])
    cluster_repr[center] = [dists[0][1], dists[0][2]]

### Getting the summary

In [77]:
summary = []
for cluster, point in cluster_repr.items():
    summary.append(summaries[point[0]])

In [78]:
summary

['Despite periodic attempts to assert itself, the legislative branch over time has settled into a secondary role to the executive branch on questions of national security. The dominance of the executive branch in the nuclear age – when presidents claimed the need to act on a moment’s notice – continued into the age of terrorism with the claimed need for vast new spy powers handed over by Congress with the Patriot Act and renewed and extended ever since. Bowing to the president in the interest of protecting the nation has been commonplace for a century, ever since the U.S. became a major international player and had to react quickly to crises. “Presidents assumed power and got away with it,” “It’s the perfect example of Congress handing over power to the executive branch and failing to keep up with it The war powers act is routinely violated these days, “In recent years Congress has tended to defer to the executive branch when it comes to national security-related issues.”',
 'As the co